Data Prep

In [ ]:
import geopandas as gpd
import xarray as xr
import numpy as np

# Read county shapefile, combo of state FIPS code and county FIPS code as multi-index
counties = gpd.read_file(
    "https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_20m.zip"
).to_crs("EPSG:3395")
counties["STATEFP"] = counties.STATEFP.astype(int)
counties["COUNTYFP"] = counties.COUNTYFP.astype(int)
continental = counties[
    ~counties["STATEFP"].isin([2, 15, 72])
]  # drop Alaska, Hawaii, Puerto Rico

# Read in saved data from xarray-water-model.py
ds = xr.open_dataset("mean_zwattablrt_nwm_1979_2020.nc")
ds["week"] = ds.time.dt.strftime("%Y-%U")
ds = ds.groupby("week").mean()
# Interpret county as combo of state FIPS code and county FIPS code
ds.coords["STATEFP"] = (ds.county // 1000).astype(int)
ds.coords["COUNTYFP"] = np.mod(ds.county, 1000).astype(int)
df = ds.to_dataframe().reset_index()

# Join
merge_df = continental.merge(df, on=["STATEFP", "COUNTYFP"])

Make all the plots

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from datetime import datetime

day_0 = datetime.strptime(weeks[0] + "-0", "%Y-%U-%w")
weeks = merge_df.week.unique()

for week in weeks:
    fig, ax = plt.subplots(1, 1, figsize=(7.68, 4.32))  # for 3840x2160 resolution

    ax.set_axis_off()

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="5%", pad=0.1)

    cax.tick_params(labelsize=8)
    cax.set_title("Depth (in meters) of the water table", fontsize=8)

    merge_df[merge_df["week"] == f"{week}"].plot(
        column="zwattablrt",
        cmap="BrBG_r",
        vmin=0,
        vmax=2,
        legend=True,
        ax=ax,
        cax=cax,
        legend_kwds={
            "orientation": "horizontal",
            "ticks": [0, 0.5, 1, 1.5, 2],
        },
    )

    # Add legends for memory, time, and cost
    current_day = datetime.strptime(week + "-0", "%Y-%U-%w")
    n_days = (current_day - day_0).days
    memory = n_days * (16.88 * 1.07374)  # daily memory converted to GB
    cost = (n_days / 7) * 0.01124606  # weekly cost

    if memory >= 1000:
        memory_string = f"{memory/1000:.1f} TB processed, ~${cost:.2f} in cloud costs"
    else:
        memory_string = f"{memory:.0f} GB processed, ~${cost:.2f} in cloud costs"

    plt.text(0, 1, memory_string, transform=ax.transAxes, size=9)
    # convert Year - Week Number to Month - Year
    date = datetime.strptime(week + "-0", "%Y-%U-%w").strftime("%Y %b")
    plt.text(0.85, 1, f"{date}", transform=ax.transAxes, size=10)
    plt.savefig(f"../../nwm-animation/3840x2160/{week}.png", transparent=True, dpi=500)
    plt.close()

Use [ffmpeg](https://ffmpeg.org/) to stitch the images together

In [ ]:
# ffmpeg -pattern_type glob -i '3840x2160/*.png' -r 60 -crf 18 -pix_fmt yuv420p nwm-video.mp4